In [1]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm


c:\LawAI\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_parquet("korean_precedents_clean.parquet")

df = df[df["text_length"] > 300]  # 너무 짧은 판례 제거
df = df.reset_index(drop=True)


In [3]:
model = SentenceTransformer("snunlp/KR-SBERT-V40K-klueNLI-augSTS")


c:\LawAI\venv\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\human-23\.cache\huggingface\hub\models--snunlp--KR-SBERT-V40K-klueNLI-augSTS. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP downlo

In [4]:
embeddings = []

for text in tqdm(df["case_text"].tolist()):
    embeddings.append(model.encode(text))

df["embedding"] = embeddings


100%|██████████| 81336/81336 [3:57:02<00:00,  5.72it/s]  


In [5]:
df.to_parquet("korean_precedents_embedded.parquet")


In [6]:
user_case = """
임차인이 임대차 기간 중 임대인의 동의 없이 내부 구조를 변경하였고,
임대인은 계약 위반을 이유로 손해배상을 청구하였다.
"""


In [7]:
selected_type = "민사"

filtered_df = df[df["사건종류명"].str.contains(selected_type, na=False)]


In [8]:
user_embedding = model.encode(user_case)

similarities = cosine_similarity(
    [user_embedding],
    list(filtered_df["embedding"])
)[0]

filtered_df["similarity"] = similarities


C:\Users\human-23\AppData\Local\Temp\ipykernel_8300\2382732111.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df["similarity"] = similarities


In [9]:
top_k = (
    filtered_df
    .sort_values("similarity", ascending=False)
    .head(5)
)

top_k[[
    "사건명",
    "법원명",
    "선고일자_norm",
    "판결유형",
    "similarity"
]]


,사건명,법원명,선고일자_norm,판결유형,similarity
80881,손해배상(기)[임대인의 철거·재건축계획의 고지행위에 관하여 종전 임차인이 권리금 회...,대법원,2022-08-11,판결,0.786839
34484,"소유권이전등기,점포명도",대법원,1993-12-07,판결,0.780843
21039,보증금청구사건,서울고법,1986-06-16,제4민사부판결 : 확정,0.777947
39504,건물명도,대법원,1996-09-06,판결,0.776552
81207,부당이득금,대법원,2023-03-30,판결,0.774808


In [10]:
import faiss
import numpy as np

In [11]:
vectors = np.vstack(df["embedding"].values).astype("float32")

In [12]:
index = faiss.IndexFlatIP(vectors.shape[1])
faiss.normalize_L2(vectors)
index.add(vectors)

In [14]:
query_vec = model.encode(user_case).astype("float32")
faiss.normalize_L2(query_vec.reshape(1, -1))

D, I = index.search(query_vec.reshape(1, -1), k=10)

In [15]:
top_cases = df.iloc[I[0]]
top_cases["similarity"] = D[0]

C:\Users\human-23\AppData\Local\Temp\ipykernel_8300\1726760079.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_cases["similarity"] = D[0]


In [17]:
def explain_similarity(row):
    return f"""
이 판례는 입력 사건과 관련하여
- 계약 관계 구조가 유사하며
- 손해배상 청구 요건 판단이 핵심 쟁점으로 다뤄졌고
- 판결 결과는 '{row["판결유형"]}'로 귀결되었습니다.
"""

In [1]:
import numpy as np
import pandas as pd
import faiss
from sentence_transformers import SentenceTransformer

# 데이터 로드
df = pd.read_parquet("korean_precedents_clean.parquet")

# 모델 (768차원)
model = SentenceTransformer("snunlp/KR-SBERT-V40K-klueNLI-augSTS")

# 임베딩 대상 텍스트
texts = df["case_text"].fillna("").tolist()

# 임베딩
embeddings = model.encode(
    texts,
    batch_size=32,
    show_progress_bar=True,
    convert_to_numpy=True
).astype("float32")

print("Embedding shape:", embeddings.shape)  # (N, 768)

# FAISS index 생성
dim = embeddings.shape[1]
index = faiss.IndexFlatL2(dim)
index.add(embeddings)

# 저장
np.save("case_embeddings.npy", embeddings)
faiss.write_index(index, "case_index.faiss")


c:\LawAI\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Batches: 100%|██████████| 2543/2543 [2:37:25<00:00,  3.71s/it]  


Embedding shape: (81368, 768)


In [5]:
test_queries = [
    {
        "query": "전세로 살던 집에서 계약 기간이 남았는데 집주인이 갑자기 나가라고 해서 분쟁이 생겼습니다.",
        "expected_keywords": ["임대차", "계약해지", "전세"]
    },
    {
        "query": "월세를 몇 달 밀렸다는 이유로 집주인이 계약을 해지하고 보증금을 안 돌려주고 있습니다.",
        "expected_keywords": ["차임연체", "임대차", "보증금"]
    },
    {
        "query": "회사에서 갑자기 해고 통보를 받았는데 사전 경고나 절차가 없었습니다.",
        "expected_keywords": ["부당해고", "근로기준법"]
    },
    {
        "query": "상사가 계속 욕설을 하고 모욕적인 발언을 해서 정신적으로 힘들었습니다.",
        "expected_keywords": ["직장 내 괴롭힘", "손해배상"]
    },
    {
        "query": "술에 취해 시비가 붙어서 상대방을 밀쳤는데 다쳤다고 고소를 당했습니다.",
        "expected_keywords": ["폭행", "형사"]
    },
    {
        "query": "주차 문제로 다투다가 상대방 차량을 발로 차서 파손했습니다.",
        "expected_keywords": ["재물손괴"]
    },
    {
        "query": "이혼을 하면서 아이 양육권을 누가 가져갈지로 다투고 있습니다.",
        "expected_keywords": ["양육권", "이혼"]
    },
    {
        "query": "이혼 후에도 전 배우자가 양육비를 전혀 지급하지 않고 있습니다.",
        "expected_keywords": ["양육비", "이행명령"]
    },
    {
        "query": "중고 거래에서 돈을 보냈는데 물건을 받지 못하고 연락이 끊겼습니다.",
        "expected_keywords": ["사기"]
    },
    {
        "query": "공사 대금을 다 지급했는데 하자가 심해서 보수를 요구하고 있습니다.",
        "expected_keywords": ["하자보수", "도급"]
    },
    {
        "query": "회사에서 초과근무를 시켰지만 야근 수당을 지급하지 않았습니다.",
        "expected_keywords": ["임금체불", "근로기준법"]
    },
    {
        "query": "친권자는 저인데 상대방이 아이를 데려가 돌려보내지 않습니다.",
        "expected_keywords": ["면접교섭", "가사"]
    }
]


In [9]:
def recall_at_k(true_id, retrieved_ids, k):
    return int(true_id in retrieved_ids[:k])

In [6]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def evaluate_recall_at_k(
    test_queries,
    model,
    faiss_index,
    df,
    k=10
):
    hits = []

    for item in test_queries:
        query = item["query"]
        keywords = item["expected_keywords"]

        q_vec = model.encode([query]).astype("float32")
        D, I = faiss_index.search(q_vec, k)

        retrieved = df.iloc[I[0]]

        hit = False
        for _, row in retrieved.iterrows():
            text = row["case_text"]
            if any(kw in text for kw in keywords):
                hit = True
                break

        hits.append(hit)

    recall = sum(hits) / len(hits)
    return recall


In [10]:
import pandas as pd
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer


In [12]:
df = pd.read_parquet(
    r"C:\LawAI\notebooks\korean_precedents_clean.parquet",
    engine="pyarrow"
)

print(df.shape)
df.head(2)


(81368, 10)


,판례정보일련번호,사건번호,사건명,법원명,사건종류명,판결유형,선고일자_norm,참조조문,case_text,text_length
0,85830,4280민상278,토지건물매매무효확인,대법원,민사,판결,1947-03-23,None,[사건명] 토지건물매매무효확인\n[판시사항] 회사소유 부동산을 회사대표자 개인이 계...,1624
1,85834,4281민상298,토지건물소유권이전등기,대법원,민사,판결,1948-04-02,None,[사건명] 토지건물소유권이전등기\n[판시사항] 잔대금 지불후에 이전등기절차를 이행하...,2853


In [14]:
faiss_index = faiss.read_index(
    r"C:\LawAI\notebooks\case_index.faiss"
)

case_embeddings = np.load(
    r"C:\LawAI\notebooks\case_embeddings.npy"
)

print("FAISS dimension:", faiss_index.d)
print("Embedding shape:", case_embeddings.shape)


FAISS dimension: 768
Embedding shape: (81368, 768)


In [15]:
model = SentenceTransformer(
    "snunlp/KR-SBERT-V40K-klueNLI-augSTS"
)


In [16]:
def evaluate_recall_at_k(
    test_queries,
    model,
    faiss_index,
    df,
    k=10
):
    hits = []

    for item in test_queries:
        query = item["query"]
        keywords = item["expected_keywords"]

        q_vec = model.encode([query]).astype("float32")
        D, I = faiss_index.search(q_vec, k)

        retrieved = df.iloc[I[0]]

        hit = False
        for _, row in retrieved.iterrows():
            text = row["case_text"]
            if any(kw in text for kw in keywords):
                hit = True
                break

        hits.append(hit)

    return sum(hits) / len(hits)


In [17]:
recall_10 = evaluate_recall_at_k(
    test_queries,
    model,
    faiss_index,
    df,
    k=10
)

print(f"Recall@10: {recall_10:.2f}")


Recall@10: 0.92
